<a href="https://colab.research.google.com/github/gorgeousbeasty/IndoNLU/blob/master/IndoBERT_Try_Out.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Load Model**
https://huggingface.co/models

untuk task classification & sequence tagging, gunakan Encoder-only model transformer

https://huggingface.co/indobenchmark/indobert-base-p1

In [6]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.0 MB/s eta 0:00:00


In [7]:
from transformers import BertForSequenceClassification, BertConfig, BertTokenizer
from nltk.tokenize import TweetTokenizer

In [8]:
pip install utils


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-linux_x86_64.whl size=4399283 sha256=6ea7e193c4a45ee984dd55cb0780491bfcf6456340c98069a367c8d9e905e514
  Stored in directory: /root/.cache/pip/wheels/93/61/2a/c54711a91c418ba06ba195b1d78ff24fcaad8592f2a694ac94
Successfully built fasttext


In [12]:
import random
import numpy as np
import pandas as pd
import torch
from torch import optim
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import fasttext
from nltk import word_tokenize

In [13]:
#clone forked IndoNLU Github
!git clone 'https://github.com/dindapusparahmi/IndoNLU/'

Cloning into 'IndoNLU'...
remote: Enumerating objects: 500, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 500 (delta 115), reused 142 (delta 111), pack-reused 316
Receiving objects: 100% (500/500), 9.45 MiB | 17.19 MiB/s, done.
Resolving deltas: 100% (235/235), done.


In [14]:
from IndoNLU.utils.forward_fn import forward_sequence_classification
from IndoNLU.utils.metrics import document_sentiment_metrics_fn
from IndoNLU.utils.data_utils import DocumentSentimentDataset, DocumentSentimentDataLoader

In [15]:
# Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p1')
config.num_labels = DocumentSentimentDataset.NUM_LABELS

# Instantiate model
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', config=config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [18]:
###
# common functions
###
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())
    
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def metrics_to_string(metric_dict):
    string_list = []
    for key, value in metric_dict.items():
        string_list.append('{}:{:.2f}'.format(key, value))
    return ' '.join(string_list)

In [19]:
count_param(model)

124443651

In [20]:
# Set random seed
set_seed(26022023)

In [21]:
train_dataset_path = '/content/IndoNLU/dataset/smsa_doc-sentiment-prosa/train_preprocess.tsv'
valid_dataset_path = '/content/IndoNLU/dataset/smsa_doc-sentiment-prosa/valid_preprocess.tsv'
test_dataset_path = '/content/IndoNLU/dataset/smsa_doc-sentiment-prosa/test_preprocess.tsv'

In [22]:
from torch.utils.data import Dataset, DataLoader

In [23]:
#####
# Document Sentiment Prosa
#####
class DocumentSentimentDataset(Dataset):
    # Static constant variable
    LABEL2INDEX = {'positive': 0, 'neutral': 1, 'negative': 2}
    INDEX2LABEL = {0: 'positive', 1: 'neutral', 2: 'negative'}
    NUM_LABELS = 3
    
    def load_dataset(self, path): 
        df = pd.read_csv(path, sep='\t', header=None)
        df.columns = ['text','sentiment']
        df['sentiment'] = df['sentiment'].apply(lambda lab: self.LABEL2INDEX[lab])
        return df
    
    def __init__(self, dataset_path, tokenizer, no_special_token=False, *args, **kwargs):
        self.data = self.load_dataset(dataset_path)
        self.tokenizer = tokenizer
        self.no_special_token = no_special_token
    
    def __getitem__(self, index):
        data = self.data.loc[index,:]
        text, sentiment = data['text'], data['sentiment']
        subwords = self.tokenizer.encode(text, add_special_tokens=not self.no_special_token)
        return np.array(subwords), np.array(sentiment), data['text']
    
    def __len__(self):
        return len(self.data)    
        
class DocumentSentimentDataLoader(DataLoader):
    def __init__(self, max_seq_len=512, *args, **kwargs):
        super(DocumentSentimentDataLoader, self).__init__(*args, **kwargs)
        self.collate_fn = self._collate_fn
        self.max_seq_len = max_seq_len
        
    def _collate_fn(self, batch):
        batch_size = len(batch)
        max_seq_len = max(map(lambda x: len(x[0]), batch))
        max_seq_len = min(self.max_seq_len, max_seq_len)
        
        subword_batch = np.zeros((batch_size, max_seq_len), dtype=np.int64)
        mask_batch = np.zeros((batch_size, max_seq_len), dtype=np.float32)
        sentiment_batch = np.zeros((batch_size, 1), dtype=np.int64)
        
        seq_list = []
        for i, (subwords, sentiment, raw_seq) in enumerate(batch):
            subwords = subwords[:max_seq_len]
            subword_batch[i,:len(subwords)] = subwords
            mask_batch[i,:len(subwords)] = 1
            sentiment_batch[i,0] = sentiment
            
            seq_list.append(raw_seq)
            
        return subword_batch, mask_batch, sentiment_batch, seq_list



In [24]:
train_dataset = DocumentSentimentDataset(train_dataset_path, tokenizer, lowercase=True)
valid_dataset = DocumentSentimentDataset(valid_dataset_path, tokenizer, lowercase=True)
test_dataset = DocumentSentimentDataset(test_dataset_path, tokenizer, lowercase=True, on_bad_lines='skip')

train_loader = DocumentSentimentDataLoader(dataset=train_dataset, max_seq_len=512, batch_size=32, num_workers=16, shuffle=True)  
valid_loader = DocumentSentimentDataLoader(dataset=valid_dataset, max_seq_len=512, batch_size=32, num_workers=16, shuffle=False)  
test_loader = DocumentSentimentDataLoader(dataset=test_dataset, max_seq_len=512, batch_size=32, num_workers=16, shuffle=False)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [25]:
w2i, i2w = DocumentSentimentDataset.LABEL2INDEX, DocumentSentimentDataset.INDEX2LABEL
print(w2i)
print(i2w)

{'positive': 0, 'neutral': 1, 'negative': 2}
{0: 'positive', 1: 'neutral', 2: 'negative'}


# **Fine Tuning & Evaluation**

In [26]:
import os, sys
sys.path.append('../')
os.chdir('../')
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [27]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [28]:
import torch

###
# Forward Function
###

# Forward function for sequence classification
def forward_sequence_classification(model, batch_data, i2w, is_test=False, device='cpu', **kwargs):
    # Unpack batch data
    if len(batch_data) == 3:
        (subword_batch, mask_batch, label_batch) = batch_data
        token_type_batch = None
    elif len(batch_data) == 4:
        (subword_batch, mask_batch, token_type_batch, label_batch) = batch_data
    
    # Prepare input & label
    subword_batch = torch.LongTensor(subword_batch)
    mask_batch = torch.FloatTensor(mask_batch)
    token_type_batch = torch.LongTensor(token_type_batch) if token_type_batch is not None else None
    label_batch = torch.LongTensor(label_batch)
            
    if device == "cuda":
        subword_batch = subword_batch.cuda()
        mask_batch = mask_batch.cuda()
        token_type_batch = token_type_batch.cuda() if token_type_batch is not None else None
        label_batch = label_batch.cuda()

    # Forward model
    outputs = model(subword_batch, attention_mask=mask_batch, token_type_ids=token_type_batch, labels=label_batch)
    loss, logits = outputs[:2]
    
    # generate prediction & label list
    list_hyp = []
    list_label = []
    hyp = torch.topk(logits, 1)[1]
    for j in range(len(hyp)):
        list_hyp.append(i2w[hyp[j].item()])
        list_label.append(i2w[label_batch[j][0].item()])
        
    return loss, list_hyp, list_label

# Forward function for word classification
def forward_word_classification(model, batch_data, i2w, is_test=False, device='cpu', **kwargs):
    # Unpack batch data
    if len(batch_data) == 4:
        (subword_batch, mask_batch, subword_to_word_indices_batch, label_batch) = batch_data
        token_type_batch = None
    elif len(batch_data) == 5:
        (subword_batch, mask_batch, token_type_batch, subword_to_word_indices_batch, label_batch) = batch_data
    
    # Prepare input & label
    subword_batch = torch.LongTensor(subword_batch)
    mask_batch = torch.FloatTensor(mask_batch)
    token_type_batch = torch.LongTensor(token_type_batch) if token_type_batch is not None else None
    subword_to_word_indices_batch = torch.LongTensor(subword_to_word_indices_batch)
    label_batch = torch.LongTensor(label_batch)

    if device == "cuda":
        subword_batch = subword_batch.cuda()
        mask_batch = mask_batch.cuda()
        token_type_batch = token_type_batch.cuda() if token_type_batch is not None else None
        subword_to_word_indices_batch = subword_to_word_indices_batch.cuda()
        label_batch = label_batch.cuda()

    # Forward model
    outputs = model(subword_batch, subword_to_word_indices_batch, attention_mask=mask_batch, token_type_ids=token_type_batch, labels=label_batch)
    loss, logits = outputs[:2]
    
    # generate prediction & label list
    list_hyps = []
    list_labels = []
    hyps_list = torch.topk(logits, k=1, dim=-1)[1].squeeze(dim=-1)
    for i in range(len(hyps_list)):
        hyps, labels = hyps_list[i].tolist(), label_batch[i].tolist()        
        list_hyp, list_label = [], []
        for j in range(len(hyps)):
            if labels[j] == -100:
                break
            else:
                list_hyp.append(i2w[hyps[j]])
                list_label.append(i2w[labels[j]])
        list_hyps.append(list_hyp)
        list_labels.append(list_label)
        
    return loss, list_hyps, list_labels

# Forward function for sequence multilabel classification
def forward_sequence_multi_classification(model, batch_data, i2w, is_test=False, device='cpu', **kwargs):
    # Unpack batch data
    if len(batch_data) == 3:
        (subword_batch, mask_batch, label_batch) = batch_data
        token_type_batch = None
    elif len(batch_data) == 4:
        (subword_batch, mask_batch, token_type_batch, label_batch) = batch_data
    
    # Prepare input & label
    subword_batch = torch.LongTensor(subword_batch)
    mask_batch = torch.FloatTensor(mask_batch)
    token_type_batch = torch.LongTensor(token_type_batch) if token_type_batch is not None else None
    label_batch = torch.LongTensor(label_batch)
            
    if device == "cuda":
        subword_batch = subword_batch.cuda()
        mask_batch = mask_batch.cuda()
        token_type_batch = token_type_batch.cuda() if token_type_batch is not None else None
        label_batch = label_batch.cuda()

    # Forward model
    outputs = model(subword_batch, attention_mask=mask_batch, token_type_ids=token_type_batch, labels=label_batch)
    loss, logits = outputs[:2] # logits list<tensor(bs, num_label)> ~ list of batch prediction per class 
    import torch

###
# Forward Function
###

# Forward function for sequence classification
def forward_sequence_classification(model, batch_data, i2w, is_test=False, device='cpu', **kwargs):
    # Unpack batch data
    if len(batch_data) == 3:
        (subword_batch, mask_batch, label_batch) = batch_data
        token_type_batch = None
    elif len(batch_data) == 4:
        (subword_batch, mask_batch, token_type_batch, label_batch) = batch_data
    
    # Prepare input & label
    subword_batch = torch.LongTensor(subword_batch)
    mask_batch = torch.FloatTensor(mask_batch)
    token_type_batch = torch.LongTensor(token_type_batch) if token_type_batch is not None else None
    label_batch = torch.LongTensor(label_batch)
            
    if device == "cuda":
        subword_batch = subword_batch.cuda()
        mask_batch = mask_batch.cuda()
        token_type_batch = token_type_batch.cuda() if token_type_batch is not None else None
        label_batch = label_batch.cuda()

    # Forward model
    outputs = model(subword_batch, attention_mask=mask_batch, token_type_ids=token_type_batch, labels=label_batch)
    loss, logits = outputs[:2]
    
    # generate prediction & label list
    list_hyp = []
    list_label = []
    hyp = torch.topk(logits, 1)[1]
    for j in range(len(hyp)):
        list_hyp.append(i2w[hyp[j].item()])
        list_label.append(i2w[label_batch[j][0].item()])
        
    return loss, list_hyp, list_label

# Forward function for word classification
def forward_word_classification(model, batch_data, i2w, is_test=False, device='cpu', **kwargs):
    # Unpack batch data
    if len(batch_data) == 4:
        (subword_batch, mask_batch, subword_to_word_indices_batch, label_batch) = batch_data
        token_type_batch = None
    elif len(batch_data) == 5:
        (subword_batch, mask_batch, token_type_batch, subword_to_word_indices_batch, label_batch) = batch_data
    
    # Prepare input & label
    subword_batch = torch.LongTensor(subword_batch)
    mask_batch = torch.FloatTensor(mask_batch)
    token_type_batch = torch.LongTensor(token_type_batch) if token_type_batch is not None else None
    subword_to_word_indices_batch = torch.LongTensor(subword_to_word_indices_batch)
    label_batch = torch.LongTensor(label_batch)

    if device == "cuda":
        subword_batch = subword_batch.cuda()
        mask_batch = mask_batch.cuda()
        token_type_batch = token_type_batch.cuda() if token_type_batch is not None else None
        subword_to_word_indices_batch = subword_to_word_indices_batch.cuda()
        label_batch = label_batch.cuda()

    # Forward model
    outputs = model(subword_batch, subword_to_word_indices_batch, attention_mask=mask_batch, token_type_ids=token_type_batch, labels=label_batch)
    loss, logits = outputs[:2]
    
    # generate prediction & label list
    list_hyps = []
    list_labels = []
    hyps_list = torch.topk(logits, k=1, dim=-1)[1].squeeze(dim=-1)
    for i in range(len(hyps_list)):
        hyps, labels = hyps_list[i].tolist(), label_batch[i].tolist()        
        list_hyp, list_label = [], []
        for j in range(len(hyps)):
            if labels[j] == -100:
                break
            else:
                list_hyp.append(i2w[hyps[j]])
                list_label.append(i2w[labels[j]])
        list_hyps.append(list_hyp)
        list_labels.append(list_label)
        
    return loss, list_hyps, list_labels

# Forward function for sequence multilabel classification
def forward_sequence_multi_classification(model, batch_data, i2w, is_test=False, device='cpu', **kwargs):
    # Unpack batch data
    if len(batch_data) == 3:
        (subword_batch, mask_batch, label_batch) = batch_data
        token_type_batch = None
    elif len(batch_data) == 4:
        (subword_batch, mask_batch, token_type_batch, label_batch) = batch_data
    
    # Prepare input & label
    subword_batch = torch.LongTensor(subword_batch)
    mask_batch = torch.FloatTensor(mask_batch)
    token_type_batch = torch.LongTensor(token_type_batch) if token_type_batch is not None else None
    label_batch = torch.LongTensor(label_batch)
            
    if device == "cuda":
        subword_batch = subword_batch.cuda()
        mask_batch = mask_batch.cuda()
        token_type_batch = token_type_batch.cuda() if token_type_batch is not None else None
        label_batch = label_batch.cuda()

    # Forward model
    outputs = model(subword_batch, attention_mask=mask_batch, token_type_ids=token_type_batch, labels=label_batch)
    loss, logits = outputs[:2] # logits list<tensor(bs, num_label)> ~ list of batch prediction per class 
    
    

In [29]:
optimizer = optim.Adam(model.parameters(), lr=5e-6)


In [31]:
model = model.cuda()

'3'

In [32]:
# Train
n_epochs = 5
for epoch in range(n_epochs):
    model.train()
    torch.set_grad_enabled(True)
 
    total_train_loss = 0
    list_hyp, list_label = [], []

    train_pbar = tqdm(train_loader, leave=True, total=len(train_loader))
    for i, batch_data in enumerate(train_pbar):
        # Forward model
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        # Update model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss = loss.item()
        total_train_loss = total_train_loss + tr_loss

        # Calculate metrics
        list_hyp += batch_hyp
        list_label += batch_label

        train_pbar.set_description("(Epoch {}) TRAIN LOSS:{:.4f} LR:{:.8f}".format((epoch+1),
            total_train_loss/(i+1), get_lr(optimizer)))

    # Calculate train metric
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) TRAIN LOSS:{:.4f} {} LR:{:.8f}".format((epoch+1),
        total_train_loss/(i+1), metrics_to_string(metrics), get_lr(optimizer)))

    # Evaluate on validation
    model.eval()
    torch.set_grad_enabled(False)
    
    total_loss, total_correct, total_labels = 0, 0, 0
    list_hyp, list_label = [], []

    pbar = tqdm(valid_loader, leave=True, total=len(valid_loader))
    for i, batch_data in enumerate(pbar):
        batch_seq = batch_data[-1]        
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
        
        # Calculate total loss
        valid_loss = loss.item()
        total_loss = total_loss + valid_loss

        # Calculate evaluation metrics
        list_hyp += batch_hyp
        list_label += batch_label
        metrics = document_sentiment_metrics_fn(list_hyp, list_label)

        pbar.set_description("VALID LOSS:{:.4f} {}".format(total_loss/(i+1), metrics_to_string(metrics)))
        
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) VALID LOSS:{:.4f} {}".format((epoch+1),
        total_loss/(i+1), metrics_to_string(metrics)))

  0%|          | 0/344 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
(Epoch 1) TRAIN LOSS:0.3082 LR:0.00000500: 100%|██████████| 344/344 [02:38<00:00,  2.18it/s]


(Epoch 1) TRAIN LOSS:0.3082 ACC:0.88 F1:0.84 REC:0.81 PRE:0.87 LR:0.00000500


  0%|          | 0/40 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
VALID LOSS:0.1809 ACC:0.93 F1:0.90 REC:0.90 PRE:0.91: 100%|██████████| 40/40 [00:08<00:00,  4.77it/s]


(Epoch 1) VALID LOSS:0.1809 ACC:0.93 F1:0.90 REC:0.90 PRE:0.91


  0%|          | 0/344 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
(Epoch 2) TRAIN LOSS:0.1408 LR:0.00000500: 100%|██████████| 344/344 [02:41<00:00,  2.12it/s]


...

(Epoch 4) VALID LOSS:0.2028 ACC:0.93 F1:0.91 REC:0.90 PRE:0.92


  0%|          | 0/344 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
(Epoch 5) TRAIN LOSS:0.0400 LR:0.00000500: 100%|██████████| 344/344 [02:42<00:00,  2.11it/s]


(Epoch 5) TRAIN LOSS:0.0400 ACC:0.99 F1:0.99 REC:0.98 PRE:0.99 LR:0.00000500


  0%|          | 0/40 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
VALID LOSS:0.2035 ACC:0.94 F1:0.91 REC:0.91 PRE:0.92: 100%|██████████| 40/40 [00:08<00:00,  4.64it/s]

(Epoch 5) VALID LOSS:0.2035 ACC:0.94 F1:0.91 REC:0.91 PRE:0.92


In [33]:
# Evaluate on test
model.eval()
torch.set_grad_enabled(False)

total_loss, total_correct, total_labels = 0, 0, 0
list_hyp, list_label = [], []

pbar = tqdm(test_loader, leave=True, total=len(test_loader))
for i, batch_data in enumerate(pbar):
    _, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
    list_hyp += batch_hyp
    list_label += batch_label
metrics = document_sentiment_metrics_fn(list_hyp, list_label)
print("TEST Metrics | {}".format(metrics_to_string(metrics)))

  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|██████████| 16/16 [00:03<00:00,  4.81it/s]

TEST Metrics | ACC:0.92 F1:0.90 REC:0.89 PRE:0.92


# **Test Fine Tuned Model on Sample Sentence**

Mencoba fine tuned model pada frasa berita yang diekstraksi
- 2 dari 3 frasa dengan label up diklasifikasi positive. 1 diklasifikasi negatif (53,81%)
- 2dari 3 frasa dengan label down diklasifikasi negatif.
- narasi berita dengan ukuran asli terlalu besar sehingga menhasilkan error

In [37]:
text = 'data penjualan Gaikindo saat kenaikan BBM yang terjadi di masa lalu. Menurut data yang disampaikan, penjualan BBM naik malah bikin penjualan naik."Tahun 2003 pernah ada kenaikan bahan bakar, 2004 naik lagi harga bbmnya. Kemudian disusul tahun 2005. Namun di sepanjang 3 tahun itu kondisinya menarik. Melihat penjualan grafik 2003, 2004, 2005 itu naik, dari ke 354 (ribu) ke 534 (ribu)," ungkap Kukuh saat diskusi virtual bersama Forum Wartawan Otomotif, Kamis (15/9/2022)."Dalam kurun waktu itu kenaikan bahan bakar tidak berpengaruh ke penjualan kendaraan bermotor," jelas dia.Lebih lanjut Kukuh juga membagikan data penjualan saat BBM mengalami kenaikan pada tahun 2012 dan 2013. Pola kenaikan penjualan juga terjadi dalam kurun waktu tersebut. Pada 2012, Gaikindo mencatat penjualan mencapai 894 ribu unit, dan setahun berikutnya naik jadi 1,16 juta unit."Kejadian ini berulang lagi di tahun 2012, 2013, di mana ada kenaikan bahan bakar. Saat itu pun kalau kita lihat pertumbuhan ekonomi bagus di 6,2 (persen) di 2012, dan sekitar 5,78 (persen) di 2013. Dua tahun itu penjualan kendaraan bermotor juga naik walaupun ada kenaikan bahan bakar," terang dia."Kita harapkan dari data-data sejarah seperti itu, kita harapkan kenaikan kali inipun tidak berpengaruh terhadap penjualan kendaraan bermotor,"'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: data penjualan Gaikindo saat kenaikan BBM yang terjadi di masa lalu. Menurut data yang disampaikan, penjualan BBM naik malah bikin penjualan naik."Tahun 2003 pernah ada kenaikan bahan bakar, 2004 naik lagi harga bbmnya. Kemudian disusul tahun 2005. Namun di sepanjang 3 tahun itu kondisinya menarik. Melihat penjualan grafik 2003, 2004, 2005 itu naik, dari ke 354 (ribu) ke 534 (ribu)," ungkap Kukuh saat diskusi virtual bersama Forum Wartawan Otomotif, Kamis (15/9/2022)."Dalam kurun waktu itu kenaikan bahan bakar tidak berpengaruh ke penjualan kendaraan bermotor," jelas dia.Lebih lanjut Kukuh juga membagikan data penjualan saat BBM mengalami kenaikan pada tahun 2012 dan 2013. Pola kenaikan penjualan juga terjadi dalam kurun waktu tersebut. Pada 2012, Gaikindo mencatat penjualan mencapai 894 ribu unit, dan setahun berikutnya naik jadi 1,16 juta unit."Kejadian ini berulang lagi di tahun 2012, 2013, di mana ada kenaikan bahan bakar. Saat itu pun kalau kita lihat pertumbuhan ekonomi bag

In [35]:
text = 'Banyuwangi menyiapkan 342 rubuha yang akan disebar ke sejumlah wilayah dengan tingkat populasi tikus tinggi. Termasuk Kecamatan Tegaldlimo yang mendapatkan alokasi 27 rubuha. Sebelumnya, di kecamatan ini sudah terdapat 60 rubuha yang tersebar di 6 desa."Dengan program ini, kita harapkan kelestarian burung hantu semakin terjaga. Diiringi dengan penurunan hama tikus sehingga produksi padi dan pendapatan petani semakin meningkat,"'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: Banyuwangi menyiapkan 342 rubuha yang akan disebar ke sejumlah wilayah dengan tingkat populasi tikus tinggi. Termasuk Kecamatan Tegaldlimo yang mendapatkan alokasi 27 rubuha. Sebelumnya, di kecamatan ini sudah terdapat 60 rubuha yang tersebar di 6 desa."Dengan program ini, kita harapkan kelestarian burung hantu semakin terjaga. Diiringi dengan penurunan hama tikus sehingga produksi padi dan pendapatan petani semakin meningkat," | Label : positive (97.313%)


In [38]:
text = 'Data internal Tokopedia mencatat terjadi kenaikan jumlah transaksi kategori otomotif sebesar hampir 1,5x lipat pada semester 1 2022 dibandingkan periode sama 2021. Audio & video mobil, ban mobil, aksesori motor, alat berat dan aksesori pengendara motor pun menjadi produk kategori otomotif yang paling diminati masyarakat setahun terakhir," kata Head of Regional Growth Expansion Tokopedia Nafisah Wulandari dalam keterangan tertulis, Rabu (31/8/2022).Ia menuturkan, industri otomotif yang masuk ke ranah digital mendapatkan sambutan positif dari masyarakat. Hal tersebut terlihat dari data internal yang dimiliki oleh Tokopedia menyebutkan sektor otomotif tumbuh positif.Secara angka, ia menyebutkan, Tokopedia Garage juga mencatat kenaikan transaksi sebesar hampir 6x lipat pada semester 1 2022 dibandingkan periode sama 2021.'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: Data internal Tokopedia mencatat terjadi kenaikan jumlah transaksi kategori otomotif sebesar hampir 1,5x lipat pada semester 1 2022 dibandingkan periode sama 2021. Audio & video mobil, ban mobil, aksesori motor, alat berat dan aksesori pengendara motor pun menjadi produk kategori otomotif yang paling diminati masyarakat setahun terakhir," kata Head of Regional Growth Expansion Tokopedia Nafisah Wulandari dalam keterangan tertulis, Rabu (31/8/2022).Ia menuturkan, industri otomotif yang masuk ke ranah digital mendapatkan sambutan positif dari masyarakat. Hal tersebut terlihat dari data internal yang dimiliki oleh Tokopedia menyebutkan sektor otomotif tumbuh positif.Secara angka, ia menyebutkan, Tokopedia Garage juga mencatat kenaikan transaksi sebesar hampir 6x lipat pada semester 1 2022 dibandingkan periode sama 2021. | Label : positive (80.585%)


In [36]:
text = 'Pemerintah mewajibkan BBM memiliki RON minimal 90 mulai 1 Januari 2023. Dengan demikian, BBM dengan kadar RON di bawah itu tak boleh lagi dijual.Saat ini masih terdapat beberapa BBM yang memiliki RON di bawah 90. Sebut saja, Premium yang dijual PT Pertamina (Persero) dengan RON 88 dan Revvo 89 yang dijual PT Vivo Energy Indonesia dengan RON 89.Corporate Secretary PT Pertamina Patra Niaga, Irto Ginting mengatakan, Premium sudah tidak lagi disalurkan di SPBU pada tahun ini."Premium sudah tidak disalurkan lagi di SPBU tahun 2022 ini," katanya kepada, seperti dikutip dari detikFinance Minggu (18/9/2022).Dia mengatakan, Pertalite merupakan BBM RON 90. Dengan demikian, pihaknya sesuai dengan kebijakan pemerintah tersebut."Kalau begitu kita masih sesuai dengan ketentuan itu. Karena RON terendah kita 90," jelasnya.Sementara, manajemen Vivo Energy Indonesia sempat buka suara terkait kelanjutan BBM Revvo 89. Vivo menjelaskan, pemerintah telah memutuskan untuk menghapus penjualan BBM beroktan rendah pada tanggal 31 Desember 2022."Untuk mematuhi kebijakan pemerintah, PT Vivo Energy Indonesia telah mengambil langkah‐langkah yang diperlukan untuk menghabiskan persediaan Revvo 89 kami pada akhir tahun ini,"'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: Pemerintah mewajibkan BBM memiliki RON minimal 90 mulai 1 Januari 2023. Dengan demikian, BBM dengan kadar RON di bawah itu tak boleh lagi dijual.Saat ini masih terdapat beberapa BBM yang memiliki RON di bawah 90. Sebut saja, Premium yang dijual PT Pertamina (Persero) dengan RON 88 dan Revvo 89 yang dijual PT Vivo Energy Indonesia dengan RON 89.Corporate Secretary PT Pertamina Patra Niaga, Irto Ginting mengatakan, Premium sudah tidak lagi disalurkan di SPBU pada tahun ini."Premium sudah tidak disalurkan lagi di SPBU tahun 2022 ini," katanya kepada, seperti dikutip dari detikFinance Minggu (18/9/2022).Dia mengatakan, Pertalite merupakan BBM RON 90. Dengan demikian, pihaknya sesuai dengan kebijakan pemerintah tersebut."Kalau begitu kita masih sesuai dengan ketentuan itu. Karena RON terendah kita 90," jelasnya.Sementara, manajemen Vivo Energy Indonesia sempat buka suara terkait kelanjutan BBM Revvo 89. Vivo menjelaskan, pemerintah telah memutuskan untuk menghapus penjualan BBM berokt

In [39]:
text = 'komitmen bersama menghentikan semua aktivitas pelayanan jasa pariwisata di Kabupaten Manggarai Barat itu tanpa ada paksaan dari pihak manapun.Aksi itu diteken di dalam MoU atau nota kesepahaman. Dalam MoU itu itu juga disebutkan bahwa para pelaku wisata akan menerima konsekuensinya jika ada yang melanggar MoU tersebut. Para pelaku wisata itu seperti pemilik kapal wisata, pemilik penyedia jasa transportasi darat, pemilik restoran, pemilik hotel, fotografer, guide, pelaku usaha kuliner.Disamping itu juga sanksi lain adalah jika ada yang melanggar MoU itu maka pelaku wisata itu harus bersedia untuk dibakar bentuk fasilitasnya'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: komitmen bersama menghentikan semua aktivitas pelayanan jasa pariwisata di Kabupaten Manggarai Barat itu tanpa ada paksaan dari pihak manapun.Aksi itu diteken di dalam MoU atau nota kesepahaman. Dalam MoU itu itu juga disebutkan bahwa para pelaku wisata akan menerima konsekuensinya jika ada yang melanggar MoU tersebut. Para pelaku wisata itu seperti pemilik kapal wisata, pemilik penyedia jasa transportasi darat, pemilik restoran, pemilik hotel, fotografer, guide, pelaku usaha kuliner.Disamping itu juga sanksi lain adalah jika ada yang melanggar MoU itu maka pelaku wisata itu harus bersedia untuk dibakar bentuk fasilitasnya | Label : negative (64.761%)


In [41]:
text = 'Arief kemudian menyoroti isu lain terkait penyediaan air minum yang saat ini belum dapat memenuhi kebutuhan masyarakat. Dia berharap PAM Jaya dapat memberikan pasokan air mencapai 100 persen kepada masyarakat."Kemudian hal lain yang menjadi isu kami PAM Jaya dari sisi penyediaan air minum saat ini belum cakupannya memenuhi dari keinginan masyarakat DKI Jakarat," ujar Arief.Arief memastikan PAM Jaya dan Pemprov DKI Jakarta akan menambah air baku agar bisa memenuhi kebutuhan masyarakat dengan target pelayanan 100 persen pada 2030. Penambahan air baku itu melalui Sistem Pengelolaan Air Minum (SPAM)."Ada beberapa strategi yang akan dilakukan Pam Jaya yang salah satunya pointing kita ada kemudian bagaimana menambah air baku yang kemudian bisa mencukupi kebutuhan masyarakat Jakarta. Kita mengenal dengan istilah SPAM," ucap Arief."Dari hal ini SPAM bertujuan untuk mempercepat kebutuhan dasar masyarakat di Provinsi DKI Jakarta adalah targetnya pelayanan 100 persen di tahun 2030," imbuhnya'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: Arief kemudian menyoroti isu lain terkait penyediaan air minum yang saat ini belum dapat memenuhi kebutuhan masyarakat. Dia berharap PAM Jaya dapat memberikan pasokan air mencapai 100 persen kepada masyarakat."Kemudian hal lain yang menjadi isu kami PAM Jaya dari sisi penyediaan air minum saat ini belum cakupannya memenuhi dari keinginan masyarakat DKI Jakarat," ujar Arief.Arief memastikan PAM Jaya dan Pemprov DKI Jakarta akan menambah air baku agar bisa memenuhi kebutuhan masyarakat dengan target pelayanan 100 persen pada 2030. Penambahan air baku itu melalui Sistem Pengelolaan Air Minum (SPAM)."Ada beberapa strategi yang akan dilakukan Pam Jaya yang salah satunya pointing kita ada kemudian bagaimana menambah air baku yang kemudian bisa mencukupi kebutuhan masyarakat Jakarta. Kita mengenal dengan istilah SPAM," ucap Arief."Dari hal ini SPAM bertujuan untuk mempercepat kebutuhan dasar masyarakat di Provinsi DKI Jakarta adalah targetnya pelayanan 100 persen di tahun 2030," imbuhny